In [ ]:
# default_exp models.binaryClassification
# default_cls_lvl 2

# Binary Horse Poo Model

> Simple model to detect HorsePoo vs noHorsePoo

In [ ]:
from nbdev.export import *
notebook2script('03_binaryHorsePoo.ipynb')